# HW5 — Problem 1: Kirchhoff for Pb + 1/2 O$_2$ → PbO

Compute the reaction enthalpy ΔH(T) and the entropy increment ΔS(T)−ΔS(298 K) using
piecewise heat capacities and Pb melting at $T_m=600$ K with $H_m=4810$ J/mol.

**Data** (J/mol/K unless noted):
- ΔH₍₂₉₈₎ = −219000 J/mol
- $c_p$(Pb,s) = 23.6 + 9.75e−3 T   (298–600 K)
- $c_p$(Pb,l) = 32.4 − 3.10e−3 T   (600–1000 K)
- $c_p$(O₂)   = 29.96 + 4.18e−3 T − 1.67e5 / T²   (298–1000 K)
- $c_p$(PbO)  = 37.9 + 26.8e−3 T   (298–1000 K)
- Pb melt: Tm = 600 K, Hm = 4810 J/mol

We evaluate ΔH(500 K), ΔH(600 K), ΔH(1000 K), and ΔS(1000 K)−ΔS(298 K).


In [1]:

import numpy as np

# Constants
dH298 = -219000.0  # J/mol
Tm = 600.0
Hm = 4810.0

def cp_Pb_s(T): return 23.6 + 9.75e-3*T
def cp_Pb_l(T): return 32.4 - 3.10e-3*T
def cp_O2(T):   return 29.96 + 4.18e-3*T - 1.67e5/(T**2)
def cp_PbO(T):  return 37.9 + 26.8e-3*T

# Analytic integral for cp = a + b T + c T^{-2}
def int_cp_poly(a,b,c, T1, T2):
    return a*(T2-T1) + 0.5*b*(T2**2 - T1**2) - c*(1.0/T2 - 1.0/T1)

# Integrals for segments
def int_cp_Pb_s(T1,T2):
    a,b,c = 23.6, 9.75e-3, 0.0
    return int_cp_poly(a,b,c,T1,T2)

def int_cp_Pb_l(T1,T2):
    a,b,c = 32.4, -3.10e-3, 0.0
    return int_cp_poly(a,b,c,T1,T2)

def int_cp_O2(T1,T2):
    a,b,c = 29.96, 4.18e-3, 1.67e5
    return int_cp_poly(a,b,c,T1,T2)

def int_cp_PbO(T1,T2):
    a,b,c = 37.9, 26.8e-3, 0.0
    return int_cp_poly(a,b,c,T1,T2)

# ΔH(T) via Kirchhoff with Pb melting step at 600 K (reactant-side transition)
def delta_H(T):
    if T <= 298.0:
        return dH298
    elif T <= Tm:
        # ΔH = ΔH298 + ∫ Δcp dT, with reactants: Pb(s) and 1/2 O2(g); products: PbO(s)
        return dH298 + (int_cp_PbO(298.0,T) - int_cp_Pb_s(298.0,T) - 0.5*int_cp_O2(298.0,T))
    else:
        # Cross the Pb melting on reactant side: subtract Hm
        part1 = (int_cp_PbO(298.0,Tm) - int_cp_Pb_s(298.0,Tm) - 0.5*int_cp_O2(298.0,Tm))
        part2 = (int_cp_PbO(Tm,T) - int_cp_Pb_l(Tm,T) - 0.5*int_cp_O2(Tm,T))
        return dH298 + part1 - Hm + part2

# ΔS increment from 298 K (reactant melting step contributes −Hm/Tm to ΔS)
def int_cp_over_T(a,b,c,T1,T2):
    # ∫ (a + bT + c/T^2) * (1/T) dT = a ln(T) + b T - c/(2 T^2)
    return a*np.log(T2/T1) + b*(T2 - T1) - c*(0.5)*(1.0/T2**2 - 1.0/T1**2)

def int_cp_over_T_Pb_s(T1,T2):
    a,b,c = 23.6, 9.75e-3, 0.0
    return int_cp_over_T(a,b,c,T1,T2)

def int_cp_over_T_Pb_l(T1,T2):
    a,b,c = 32.4, -3.10e-3, 0.0
    return int_cp_over_T(a,b,c,T1,T2)

def int_cp_over_T_O2(T1,T2):
    a,b,c = 29.96, 4.18e-3, 1.67e5
    return int_cp_over_T(a,b,c,T1,T2)

def int_cp_over_T_PbO(T1,T2):
    a,b,c = 37.9, 26.8e-3, 0.0
    return int_cp_over_T(a,b,c,T1,T2)

def delta_S_increment(T):
    if T <= 298.0:
        return 0.0
    elif T <= Tm:
        return (int_cp_over_T_PbO(298.0,T)
                - int_cp_over_T_Pb_s(298.0,T)
                - 0.5*int_cp_over_T_O2(298.0,T))
    else:
        part1 = (int_cp_over_T_PbO(298.0,Tm)
                 - int_cp_over_T_Pb_s(298.0,Tm)
                 - 0.5*int_cp_over_T_O2(298.0,Tm))
        part2 = (int_cp_over_T_PbO(Tm,T)
                 - int_cp_over_T_Pb_l(Tm,T)
                 - 0.5*int_cp_over_T_O2(Tm,T))
        # Entropy step for melting reactant Pb: −Hm/Tm
        return part1 - Hm/Tm + part2

# Evaluate and print
for T in [500.0, 600.0, 1000.0]:
    print(f"ΔH({T:.0f} K) = {delta_H(T)/1000:.3f} kJ/mol")
print(f"ΔS(1000 K) - ΔS(298 K) = {delta_S_increment(1000.0):.3f} J/mol/K")


ΔH(500 K) = -218.045 kJ/mol
ΔH(600 K) = -217.318 kJ/mol
ΔH(1000 K) = -217.076 kJ/mol
ΔS(1000 K) - ΔS(298 K) = 1.878 J/mol/K
